In [2]:
import csv
import io
import logging
import re
from pathlib import Path
from zipfile import ZipFile
from dataclasses import make_dataclass
from datetime import datetime as dt

import requests
import pandas as pd
from pyspark.shell import *
from pyspark.sql import DataFrame, functions as F, types as T

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.4
      /_/

Using Python version 3.7.3 (default, Jul  1 2019 21:52:21)
SparkSession available as 'spark'.


In [3]:
log = logging.getLogger("schemas")
logging.basicConfig(level=logging.INFO)

In [4]:
ddl = Path("./samples/landing/Desc/Layout_and_Length.txt")
samples = Path("./samples/landing/2019/")

In [39]:
dictionary = re.findall(r"(\w+)\s+(\w+)\s+(\d+)", ddl.read_text())

In [40]:
zips = {i.stem: i for i in samples.rglob("*.zip")}

In [41]:
def write_schema(fp: str, df: DataFrame) -> dict:
    schema = df.schema.json()
    with Path(fp).open("w") as f:
        log.info(f.write(schema))
    return dict(schema)

In [68]:
def tables() -> set:
    return set(i[0] for i in dictionary)

def get_fields(table: str):
    return [i[1] for i in dictionary if table.rstrip(".txt") in i[0]]

In [71]:
ZipFile(zips["Hearing_files"]).namelist()

['arb_hearings_pp.txt',
 'arb_hearings_real.txt',
 'arb_protest_pp.txt',
 'arb_protest_real.txt']

In [113]:
def extract():
    table = "arb_protest_real.txt"
    with ZipFile(zips["Hearing_files"]) as zip_file:
        with zip_file.open(table) as f:
            reader = csv.DictReader(io.TextIOWrapper(f, encoding="iso-8859-1"), fieldnames=get_fields(table), dialect="excel-tab")
            yield from reader

In [183]:
raw = [*extract()]

In [132]:
df = spark.createDataFrame(raw)

In [133]:
df.cache()

DataFrame[acct: string, protested_by: string, protested_dt: string]

In [165]:
df = df.withColumn("protested_dt", F.to_date(df.protested_dt, "MM/dd/yyyy"))

In [177]:
df.describe().show()

+-------+--------------------+------------------+
|summary|                acct|      protested_by|
+-------+--------------------+------------------+
|  count|              424219|            424219|
|   mean|1.013531628362041...| 8243.088316189662|
| stddev|5.084166816570528...|14324.128152197953|
|    min|       0010020000001|              1000|
|    max|       9700000000278|             Owner|
+-------+--------------------+------------------+



In [178]:
df.schema.jsonValue()

{'type': 'struct',
 'fields': [{'name': 'acct',
   'type': 'string',
   'nullable': True,
   'metadata': {}},
  {'name': 'protested_by', 'type': 'string', 'nullable': True, 'metadata': {}},
  {'name': 'protested_dt', 'type': 'date', 'nullable': True, 'metadata': {}}]}